In [47]:
import argparse
import cv2
import albumentations as A
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from albumentations.core.transforms_interface import ImageOnlyTransform
import yaml
from types import SimpleNamespace
import torch
import random
import numpy as np

In [48]:
class Morphological(ImageOnlyTransform):
    def __init__(self, scale=(1, 3), operation="dilation", always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.scale = scale
        self.operation = operation

    def apply(self, img, **params):
        # 커널 크기 랜덤 샘플링
        k = np.random.randint(self.scale[0], self.scale[1] + 1)
        kernel = np.ones((k, k), np.uint8)
        
        if self.operation == "dilation":
            return cv2.dilate(img, kernel, iterations=1)
        elif self.operation == "erosion":
            return cv2.erode(img, kernel, iterations=1)
        else:
            raise ValueError(f"Unsupported operation: {self.operation}")

    def get_transform_init_args_names(self):
        return ("scale", "operation")

A.Morphological = Morphological

AUG = {
    'eda': A.Compose([
        # Brightness, Contrast, ColorJitter
        A.ColorJitter(brightness=0.1, contrast=0.07, saturation=0.07, hue=0.07, p=1.0),
        # 공간 변형에 대한 증강
        A.Affine(
            # scale=(0.85, 1.15),
            translate_percent=(-0.05,0.05),
            rotate=(-20,30),
            cval=(255,255,255),
            # shear=(-5, 5),
            p=1.0
        ),
        # x,y 좌표 반전 
        A.OneOf([
            A.Compose([
                A.HorizontalFlip(p=1.0),
                A.VerticalFlip(p=1.0),
            ]),
            A.Transpose(p=1),
        ], p=0.8),
        # Blur & Noise
        A.OneOf([
            A.GaussianBlur(sigma_limit=(0.5, 2.5), p=1.0),
            A.Blur(blur_limit=(3, 9), p=1.0),
        ], p=1.0),
        A.GaussNoise(var_limit=(0.0025, 0.2), p=1.0),            
    ]),
    'dilation': A.Compose([
        A.Morphological(p=1, scale=(1, 3), operation="dilation"),
        # 공간 변형에 대한 증강
        A.Affine(
            scale=(0.85, 1.15),
            translate_percent=(-0.05,0.05),
            rotate=(-20,30),
            cval=(255,255,255),
            # shear=(-5, 5),
            p=0.9
        ),
        # x,y 좌표 반전 
        A.OneOf([
            A.Compose([
                A.HorizontalFlip(p=1.0),
                A.VerticalFlip(p=1.0),
            ]),
            A.Transpose(p=1),
        ], p=0.8),    
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=1),
        A.RandomBrightnessContrast(p=1),
    ]),
    'erosion': A.Compose([
        A.Morphological(p=1, scale=(2, 4), operation="erosion"),
        # 공간 변형에 대한 증강
        A.Affine(
            scale=(0.85, 1.15),
            translate_percent=(-0.05,0.05),
            rotate=(-20,30),
            cval=(255,255,255),
            # shear=(-5, 5),
            p=0.9
        ),
        # x,y 좌표 반전 
        A.OneOf([
            A.Compose([
                A.HorizontalFlip(p=1.0),
                A.VerticalFlip(p=1.0),
            ]),
            A.Transpose(p=1),
        ], p=0.8),   
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=1),
        A.RandomBrightnessContrast(p=1),
    ]),
    'easiest': A.Compose([
        A.Rotate(
            limit=(-20, 30),
            value=(255,255,255),
            p=1.0, # 50% 확률로 적용
        ),
        # x,y 좌표 반전 
        A.OneOf([
            A.Compose([
                A.HorizontalFlip(p=1.0),
                A.VerticalFlip(p=1.0),
            ]),
            A.Transpose(p=1),
        ], p=0.8),   
    ]),
    'stilleasy': A.Compose([
        A.Affine(
            # scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # X, Y 축 개별 스케일
            translate_percent=(-0.15, 0.15), # X, Y 축 개별 이동
            rotate=(-15, 20), # 회전 각도
            # shear=(-10, 10),  # 전단 변환 (이미지를 기울임)
            cval=(255,255,255), # 이미지 외부 = 흰색으로 채우기
            p=1.0, # 50% 확률로 적용
        ), 
        # x,y 좌표 반전 > 100% 글자 반전
        A.OneOf([
            A.HorizontalFlip(p=1.0),
            A.VerticalFlip(p=1.0),
            A.Transpose(p=1),
        ], p=0.8),
    ]),
    'basic': A.Compose([ ### 색조/밝기/대비 변화를 최소화하고 기하학전 변환에 초점을 둔 약한 증강. 노이즈/블러도 없음.
        # 1. 픽셀 값 기반 변환 (이미지 자체의 픽셀 값에 영향을 줌)
        # 이 변환들은 기하학적 변환 전에 적용하는 것이 좋습니다.
        A.RGBShift(
            r_shift_limit=20,  # Red 채널 최대 변화량 (-20 ~ +20)
            g_shift_limit=20,  # Green 채널 최대 변화량 (-20 ~ +20)
            b_shift_limit=20,  # Blue 채널 최대 변화량 (-20 ~ +20)
            p=0.5 # 50% 확률로 적용
        ),
        A.RandomBrightnessContrast(
            brightness_limit=0.2, # 밝기 변화량 (-0.2 ~ +0.2)
            contrast_limit=0.2,   # 대비 변화량 (-0.2 ~ +0.2)
            p=0.5 # 50% 확률로 적용
        ),
        
        # 2. 기하학적 변환 (이미지의 형태, 위치, 크기에 영향을 줌)
        # 이 변환들은 픽셀 기반 변환 이후에 적용하여 일관성을 유지하는 것이 좋습니다.
        # Transpose는 이미지를 전치(transpose)합니다 (행과 열을 바꿉니다).
        # 이는 이미지의 90도 회전과 대칭 조합과 유사하게 작동합니다.
        # ShiftScaleRotate와 함께 사용하면 더 다양한 방향의 변형을 줄 수 있습니다.
        A.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # X, Y 축 개별 스케일
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # X, Y 축 개별 이동
            rotate=(-15, 20), # 회전 각도
            # shear=(-10, 10),  # 전단 변환 (이미지를 기울임)
            p=0.5, # 50% 확률로 적용
            cval=(255,255,255) # 이미지 외부 = 흰색으로 채우기
        ),
        # x,y 좌표 반전 
        A.OneOf([
            A.Compose([
                A.HorizontalFlip(p=1.0),
                A.VerticalFlip(p=1.0),
            ]),
            A.Transpose(p=1),
        ], p=0.8),          
    ]),
    'middle': A.Compose([ # 노이즈/블러 + 기하학적 변환에 초점을 둔 중간 난이도의 변환
        
        # 노이즈 효과 (둘 중 하나만 적용, 문서 품질 저하를 시뮬레이션)
        A.OneOf([
            A.GaussNoise(var_limit=(0.01, 0.2), p=1.0),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1.0)
        ], p=0.6), # 노이즈도 너무 강하면 인식 어렵기에 적당한 확률 (0.2 유지)

        # 2. 기하학적 변환 및 문서 특화 변형 (형태 왜곡, 시점 변화)
        # Perspective, GridDistortion, ElasticTransform은 강한 비선형 변환이므로
        # OneOf로 묶거나 각자의 확률을 낮춰 과도한 왜곡을 방지합니다.
        # 여기서는 문서의 "찌그러짐/왜곡"을 시뮬레이션하기 위해 OneOf로 묶는 것이 효과적입니다.
        A.OneOf([
            # 문서 원근 변환 > 아주 미세하게만 변화
            A.Perspective(scale=(0.02, 0.04), pad_val=(255,255,255), p=1.0), 
            # 그리드 왜곡 (num_steps=5, distort_limit=0.1 적절)
            A.GridDistortion(num_steps=5, distort_limit=0.2, p=1.0),
        ], p=0.3), # 이 세 가지 강한 왜곡 중 하나를 30% 확률로 적용 (개별 p값이 1.0이므로 OneOf의 p가 중요)
        # 원본에서 각 p=0.3, 0.2, 0.1로 각각 적용되었으나, 이제는 OneOf로 묶어 총 적용 확률을 0.3으로 설정했습니다.
        # 이렇게 하면 세 가지 중복되는 왜곡 효과를 동시에 얻는 경우를 줄여줍니다.
        
        # 기본 기하학적 변환 (Shift, Scale, Rotate)
        # 문서의 경우 회전 제한이 중요 (원본에서 min(config.rotation_limit, 15)로 제한)
        A.Affine(
            # scale=(0.8, 1.2),
            translate_percent=(-0.25, 0.25),
            rotate=(-120, 120), # 회전 각도
            # shear=(-5, 5),  # 전단 변환 (이미지를 기울임)
            p=1.0, 
            cval=(255,255,255) # 이미지 외부 = 흰색으로 채우기
        ),

        # x,y 좌표 반전 > 100% 글자 반전
        A.OneOf([
            A.HorizontalFlip(p=1.0),
            A.VerticalFlip(p=1.0),
            A.Transpose(p=1),
        ], p=0.8),
    ]),
    'aggressive': A.Compose([
        # 정보 가리기 및 혼합 (Occlusion & Mixing)
        # Train의 마스킹과 유사한 효과를 주어 모델이 특정 영역에 의존하지 않도록함
        A.CoarseDropout(
            min_holes=3,
            max_holes=5,
            min_height=10,
            max_height=35,
            min_width=5,
            max_width=45,
            fill_value=(0,0,0),
            p=0.9
        ),
        # 강력한 기하학적 변환
        A.OneOf([
            # Affine의 범위를 크게 늘리고, Perspective 변환을 추가하여 왜곡 시뮬레이션
            A.Affine(
                scale=(0.7, 1.3),
                translate_percent=(-0.15,0.2),
                rotate=(-45, 45), # 회전 각도
                shear=(-10, 10),  # 전단 변환 (이미지를 기울임)
                p=1.0, # 50% 확률로 적용
                cval=(255,255,255) # 이미지 외부 = 흰색으로 채우기
            ),
            A.Perspective(scale=(0.05, 0.1),pad_val=(255,255,255),p=1.0),
        ], p=0.9),
        # x,y 좌표 반전 
        A.OneOf([
            A.Compose([
                A.HorizontalFlip(p=1.0),
                A.VerticalFlip(p=1.0),
            ]),
            A.Transpose(p=1),
        ], p=0.8),
        # 노이즈 효과 (둘 중 하나만 적용, 문서 품질 저하를 시뮬레이션)
        A.OneOf([
            A.GaussNoise(var_limit=(0.01, 0.3), p=1.0), 
            A.ISONoise(color_shift=(0.01, 0.2), intensity=(0.1, 0.5), p=1.0)
        ], p=0.3), # 노이즈도 너무 강하면 인식 어렵기에 적당한 확률 
        A.OneOf([
            # 스캔/촬영 시 발생할 수 있는 블러 효과
            A.GaussianBlur(blur_limit=(3, 7), p=1.0),
            A.MotionBlur(blur_limit=(3, 7), p=1.0),
            # 이미지 품질을 낮춰 압축/해상도 저하 효과 모방
            A.Downscale(
                scale_min=0.5,
                scale_max=0.75,
                p=1.0
            ),
        ], p=0.5),

        # 색상 및 대비의 급격한 변화
        A.OneOf([
            # 문서의 조명, 스캔 품질 변화 모방
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1.0),
            A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=1.0),
            # 히스토그램 평활화를 통해 대비를 극적으로 변경
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0),
        ], p=0.8),
    ]),
}


/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


In [49]:
def augment_offline_dataset(cfg, train_df, val_df, train_targets_goal, val_targets_goal):
    """
    train_targets_goal, val_targets_goal: 각 데이터셋별 클래스당 목표 이미지 수 (정수)
    
    train_df, val_df: 원본 train, val 데이터프레임
    
    증강 이미지들은 aug_train, aug_val 폴더에 저장되고,
    증강 후 데이터프레임도 반환
    """
    data_dir = "/data/ephemeral/home/upstageailab-cv-classification-cv_5/data"
    
    # 증강된 모든 이미지 저장 폴더 생성
    aug_train_dir = os.path.join(data_dir, "aug_train")
    aug_val_dir = os.path.join(data_dir, "aug_val")
    os.makedirs(aug_train_dir, exist_ok=True)
    os.makedirs(aug_val_dir, exist_ok=True)
    
    # 증강 기법 리스트
    offline_aug_policies = ['eda', 'easiest', 'stilleasy', 'middle']
    
    # 새로운 증강된 ID를 생성하기 위한 카운터 초기화
    aug_id_counters = {}
    
    # 동적 이미지명 설정 함수
    def get_new_aug_id(orig_id, aug_name, split_tag):
        base = f"{split_tag}_{aug_name}_{orig_id.split('.')[0]}"
        cnt = aug_id_counters.get(base, 0) + 1
        aug_id_counters[base] = cnt
        return f"{base}_{cnt}"

    # 데이터 오프라인 증강 함수
    def augment_class_subset(df, target_goal, save_dir, split_tag):
        augmented_ids = []
        augmented_labels = []
        
        # 사용하려는 증강 기법 개수
        n_policies = len(offline_aug_policies)

        for cls in df['target'].unique():
            cls_df = df[df['target'] == cls].reset_index(drop=True)
            current_count = len(cls_df)
            print("\n현재 개수:", current_count)
            to_generate = target_goal - current_count
            if to_generate <= 0:
                continue
            print("목표 증강 개수:", to_generate)

            # 각 증강별 목표 개수 (균등 분할)
            base_quota = to_generate // n_policies
            remainder = to_generate % n_policies

            # 각 증강별 진행 예정 횟수 저장
            aug_counts = [base_quota] * n_policies
            for i in range(remainder):
                aug_counts[i] += 1

            # 이 클래스의 증강한 이미지 수
            total_augmented_count = 0

            for aug_idx, aug_name in enumerate(offline_aug_policies):
                # 이번 증강의 증강 진행 예정 횟수
                n_aug = aug_counts[aug_idx]
                if n_aug == 0:
                    continue
                
                # 이 클래스의 증강한 이미지 수
                augmented_count = 0

                sampled_df = cls_df.sample(n=n_aug, replace=True, random_state=cfg.random_seed).reset_index(drop=True)

                aug_transform = AUG.get(aug_name)
                if aug_transform is None:
                    raise ValueError(f"'{aug_name}' 증강이 AUG 딕셔너리에 정의되어있지 않습니다.")

                for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc=f"Augment class {cls} with {aug_name}"):
                    orig_id = row['ID']
                    orig_path = os.path.join(data_dir, 'train', orig_id)

                    img = cv2.imread(orig_path)
                    if img is None:
                        print(f"⚠️ 이미지 로드 실패: {orig_path}")
                        continue

                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    augmented = aug_transform(image=img)
                    aug_img = augmented['image']
                    aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

                    new_id = get_new_aug_id(orig_id, aug_name, split_tag=save_dir.split('/')[-1]) + '.jpg'
                    save_path = os.path.join(save_dir, new_id)
                    cv2.imwrite(save_path, aug_img)

                    augmented_ids.append(new_id)
                    augmented_labels.append(cls)
                    augmented_count += 1
                
                total_augmented_count += augmented_count

                # # 혹시나 증강이 부족하다면 추가로 진행
                # if augmented_count < to_generate:
                #     shortage = to_generate - augmented_count
                #     for _ in range(shortage):
                #         aug_name = random.choice(offline_aug_policies)
                #         row = cls_df.sample(n=1, replace=True, random_state=cfg.random_seed).iloc[0]
                #         orig_id = row['ID']
                #         orig_path = os.path.join(data_dir, 'train', orig_id)

                #         img = cv2.imread(orig_path)
                #         if img is None:
                #             continue

                #         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                #         aug_transform = AUG.get(aug_name)
                #         if aug_transform is None:
                #             raise ValueError(f"Augmentation '{aug_name}' not found in AUG dict.")

                #         augmented = aug_transform(image=img)
                #         aug_img = augmented['image']
                #         aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

                #         new_id = get_new_aug_id(orig_id) + '.jpg'
                #         save_path = os.path.join(save_dir, new_id)
                #         cv2.imwrite(save_path, aug_img)

                #         augmented_ids.append(new_id)
                #         augmented_labels.append(cls)
                #         augmented_count += 1

                print(f"{cls} 클래스 : 총 {augmented_count} 개의 이미지 {offline_aug_policies[aug_idx]} 증강\n")
            print(f"{cls} 클래스 : 총 {total_augmented_count} 개의 이미지 증강됨\n")

        return augmented_ids, augmented_labels

# train, val 각각 증강 수행
    train_aug_ids, train_aug_labels = augment_class_subset(train_df, train_targets_goal, aug_train_dir, split_tag='train')
    val_aug_ids, val_aug_labels = augment_class_subset(val_df, val_targets_goal, aug_val_dir, split_tag='val')
    
    # 기존 원본 이미지도 각 폴더로 복사 (덮어쓰기 없이)
    import shutil
    
    def copy_original_images(df, save_dir):
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Copy original images to {save_dir}"):
            src = os.path.join(data_dir, 'train', row['ID'])
            dst = os.path.join(save_dir, row['ID'])
            if not os.path.exists(dst):
                shutil.copy2(src, dst)
    
    copy_original_images(train_df, aug_train_dir)
    copy_original_images(val_df, aug_val_dir)
    
    # 데이터프레임 생성 및 반환 (기존 + 증강)
    train_aug_df = pd.DataFrame({"ID": train_aug_ids, "target": train_aug_labels})
    val_aug_df = pd.DataFrame({"ID": val_aug_ids, "target": val_aug_labels})
    
    train_df_aug = pd.concat([train_df, train_aug_df], ignore_index=True).reset_index(drop=True)
    val_df_aug = pd.concat([val_df, val_aug_df], ignore_index=True).reset_index(drop=True)
    
    # CSV 저장
    train_df_aug.to_csv(os.path.join(data_dir, "train_plus_augmented.csv"), index=False)
    val_df_aug.to_csv(os.path.join(data_dir, "val_plus_augmented.csv"), index=False)
    
    print(f"✅ 오프라인 증강 완료")
    print(f"✅ 목표 훈련 데이터 수 : {train_targets_goal * 17}  증강 완료된 훈련 데이터 수 : {len(train_df_aug)}")
    print(f"✅ 목표 검증 데이터 수 : {val_targets_goal * 17}  증강 완료된 검증 데이터 수 : {len(val_df_aug)}")

In [50]:
def load_config(config_path='./config.yaml'):
    """.yaml 설정 파일 읽기

    :param str config_path: _description_, defaults to './config.yaml'
    :return _type_: _description_
    """
    with open(config_path, 'r') as file:
        cfg = yaml.safe_load(file)
    return SimpleNamespace(**cfg)

In [51]:
def set_seed(seed: int = 256):
    """랜덤성 제어 함수

    :param int seed: _description_, defaults to 256
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [52]:
# python 파일 실행할 때 config.yaml 파일 이름을 입력받아서 설정 파일을 지정한다.
parser = argparse.ArgumentParser(description="Run deep learning training with specified configuration.")
parser.add_argument(
    '--config',
    type=str,
    default='config_v2.yaml',  # 기본값
    help='Name of the configuration YAML file (e.g., config.yaml, experiment_A.yaml)'
)

args, unknown = parser.parse_known_args()  # ← 여기만 수정

# Yaml 파일 읽기
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    base_dir = os.getcwd()

config_path = os.path.join(base_dir, args.config)
cfg = load_config(config_path=config_path)

# 랜덤성 제어
set_seed(cfg.random_seed)

In [53]:
data_dir = "/data/ephemeral/home/upstageailab-cv-classification-cv_5/data"

df = pd.read_csv(os.path.join(data_dir, "train0705a.csv"))
# Train-validation 분할
train_df, val_df = train_test_split(df, test_size=cfg.val_split_ratio, random_state=cfg.random_seed, stratify=df['target'] if cfg.stratify else None)

# 전체 목표 샘플 수
target_per_class = cfg.offline_aug['max_samples']

# train/val 분할 비율
train_ratio = 1 - cfg.val_split_ratio
val_ratio = cfg.val_split_ratio

train_targets_goal = int(target_per_class * train_ratio)  # 예: 500 * 0.85 = 425
val_targets_goal = int(target_per_class * val_ratio)      # 예: 500 * 0.15 = 75

# 오프라인 증강 수행 (함수에 목표 정수 전달)
if hasattr(cfg, 'offline_aug') and cfg.offline_aug:
    augment_offline_dataset(cfg, train_df, val_df, train_targets_goal, val_targets_goal)
else:
    print("⚠️ 오프라인 증강을 수행하지 않습니다.")


현재 개수: 85
목표 증강 개수: 1615


Augment class 5 with eda:   0%|          | 0/404 [00:00<?, ?it/s]

Augment class 5 with eda: 100%|██████████| 404/404 [00:10<00:00, 37.29it/s]


5 클래스 : 총 404 개의 이미지 eda 증강



Augment class 5 with easiest: 100%|██████████| 404/404 [00:01<00:00, 230.18it/s]


...

1 클래스 : 총 73 개의 이미지 stilleasy 증강



Augment class 1 with middle: 100%|██████████| 73/73 [00:01<00:00, 49.83it/s]


1 클래스 : 총 73 개의 이미지 middle 증강

1 클래스 : 총 293 개의 이미지 증강됨



Copy original images to /data/ephemeral/home/upstageailab-cv-classification-cv_5/data/aug_train: 100%|██████████| 1334/1334 [00:00<00:00, 5679.31it/s]
Copy original images to /data/ephemeral/home/upstageailab-cv-classification-cv_5/data/aug_val: 100%|██████████| 236/236 [00:00<00:00, 4961.93it/s]


✅ 오프라인 증강 완료
✅ 목표 훈련 데이터 수 : 28900  증강 완료된 훈련 데이터 수 : 28900
✅ 목표 검증 데이터 수 : 5100  증강 완료된 검증 데이터 수 : 5100


In [54]:
df1 = pd.read_csv(os.path.join(data_dir, "train_plus_augmented.csv"))
df2 = pd.read_csv(os.path.join(data_dir, "val_plus_augmented.csv"))

In [55]:
df_merged = pd.concat([df1, df2], ignore_index=True)
df_merged = df_merged.drop_duplicates()

In [56]:
print(f"✅ df_merged shape: {df_merged.shape}")
print(f"✅ df_merged row count: {len(df_merged)}")

✅ df_merged shape: (34000, 2)
✅ df_merged row count: 34000


In [59]:
df_merged.to_csv(os.path.join(data_dir, "aug_data_2000_new1/train.csv"), index=False)

In [57]:
import os
import shutil
from tqdm import tqdm

def simple_merge_and_rename(train_dir, val_dir):
    # val_dir의 파일을 train_dir로 이동
    val_files = [f for f in os.listdir(val_dir) if os.path.isfile(os.path.join(val_dir, f))]
    for f in tqdm(val_files, desc="Moving aug_val -> aug_train"):
        src = os.path.join(val_dir, f)
        dst = os.path.join(train_dir, f)
        shutil.move(src, dst)

    # 빈 val_dir 삭제
    os.rmdir(val_dir)

aug_train_dir = os.path.join(data_dir, "aug_train")
aug_val_dir = os.path.join(data_dir, "aug_val")

simple_merge_and_rename(aug_train_dir, aug_val_dir)


Moving aug_val -> aug_train:   0%|          | 0/5100 [00:00<?, ?it/s]

Moving aug_val -> aug_train: 100%|██████████| 5100/5100 [00:00<00:00, 39278.13it/s]


In [58]:
import os

file_count = len([
    f for f in os.listdir(aug_train_dir)
    if os.path.isfile(os.path.join(aug_train_dir, f))
])

print(f"📂 '{aug_train_dir}' 안의 파일 수: {file_count}개")

📂 '/data/ephemeral/home/upstageailab-cv-classification-cv_5/data/aug_train' 안의 파일 수: 34000개
